# **Preprocessing for basic models**
> [!IMPORTANT]
> RUN THE ``basic_eda.ipynb`` FIRST

In [15]:
import pandas as pd
df = pd.read_pickle('df_for_basic.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20777 entries, 0 to 20812
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   link              20777 non-null  object
 1   publication_date  20777 non-null  object
 2   title             20777 non-null  object
 3   content           20777 non-null  object
 4   main_tag          20777 non-null  object
 5   source            20777 non-null  object
 6   content_length    20777 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 1.3+ MB


In [16]:
# standardize data to unicode
df['content_standardized'] = df['content'].str.normalize('NFC')
df['main_tag_standardized'] = df['main_tag'].str.normalize('NFC')

print("Content:")
print(df['content_standardized'])
print("Main tag:")
print(df['main_tag_standardized'])

Content:
0        Trưa 4.11, quyền Chủ tịch Liên đoàn Bóng đá Ma...
1        Vụ việc FIFA bác đơn kháng cáo và giữ nguyên á...
2        Nhà phê bình bóng đá Datuk Pekan Ramli đã bày ...
3        Cựu tiền đạo Safee Sali - biểu tượng một thời ...
4        Theo nhà báo Zulhelmi Zainal Azam của Astro Ar...
                               ...                        
20808    Hôm qua, Campuchia và Thái Lan xác nhận các tư...
20809    Không quân Ukraine hôm 9.6 cho biết Nga triển ...
20810    Hôm nay (9.6), Bộ Ngoại giao Thái Lan xác nhận...
20811    Thống đốc California Gavin Newsom tuyên bố ngà...
20812    Hôm nay (9.6), Điện Kremlin gọi kế hoạch của T...
Name: content_standardized, Length: 20777, dtype: object
Main tag:
0        Thể thao
1        Thể thao
2        Thể thao
3        Thể thao
4        Thể thao
           ...   
20808    Thế giới
20809    Thế giới
20810    Thế giới
20811    Thế giới
20812    Thế giới
Name: main_tag_standardized, Length: 20777, dtype: object


In [17]:
df['content_cleaned'] = df['content_standardized'].str.lower()
df['main_tag_cleaned'] = df['main_tag_standardized'].str.lower()

print("Content:")
print(df['content_cleaned'])
print('Main tag:')
print(df['main_tag_cleaned'])

Content:
0        trưa 4.11, quyền chủ tịch liên đoàn bóng đá ma...
1        vụ việc fifa bác đơn kháng cáo và giữ nguyên á...
2        nhà phê bình bóng đá datuk pekan ramli đã bày ...
3        cựu tiền đạo safee sali - biểu tượng một thời ...
4        theo nhà báo zulhelmi zainal azam của astro ar...
                               ...                        
20808    hôm qua, campuchia và thái lan xác nhận các tư...
20809    không quân ukraine hôm 9.6 cho biết nga triển ...
20810    hôm nay (9.6), bộ ngoại giao thái lan xác nhận...
20811    thống đốc california gavin newsom tuyên bố ngà...
20812    hôm nay (9.6), điện kremlin gọi kế hoạch của t...
Name: content_cleaned, Length: 20777, dtype: object
Main tag:
0        thể thao
1        thể thao
2        thể thao
3        thể thao
4        thể thao
           ...   
20808    thế giới
20809    thế giới
20810    thế giới
20811    thế giới
20812    thế giới
Name: main_tag_cleaned, Length: 20777, dtype: object


In [18]:
# use pyvi to tokenize the content
from pyvi import ViTokenizer

def tokenize_pyvi(text):
    if not isinstance(text, str) or not text:
        return ""
    return ViTokenizer.tokenize(text)

df['content_tokenized'] = df['content_cleaned'].apply(tokenize_pyvi)
df['main_tag_tokenized'] = df['main_tag_cleaned'].apply(tokenize_pyvi)

print(df[['content_tokenized', 'main_tag_tokenized']].head())


                                   content_tokenized main_tag_tokenized
0  trưa 4.11 , quyền chủ_tịch liên_đoàn bóng_đá m...           thể_thao
1  vụ_việc fifa bác đơn kháng_cáo và giữ nguyên á...           thể_thao
2  nhà phê_bình bóng_đá datuk pekan ramli đã bày_...           thể_thao
3  cựu tiền_đạo safee sali - biểu_tượng một thời ...           thể_thao
4  theo nhà_báo zulhelmi zainal azam của astro ar...           thể_thao


In [19]:
# removing stopwords
stopwords_filepath = './vietnamese-stopwords-dash.txt'

with open(stopwords_filepath, 'r', encoding='utf-8') as f:
    stopwords_list = f.read().splitlines()

stopwords_set = set(word.strip() for word in stopwords_list)

def remove_stopwords(tokenized_text):
    if not isinstance(tokenized_text, str):
        return ""
    
    words = tokenized_text.split()

    filtered_words = [word for word in words if word not in stopwords_set]

    return " ".join(filtered_words)

df['content_final'] = df['content_tokenized'].apply(remove_stopwords)

print('Tokenized content:\n')
print(df['content_tokenized'][0])
print('\nFinal content:\n')
print(df['content_final'][0])

Tokenized content:

trưa 4.11 , quyền chủ_tịch liên_đoàn bóng_đá malaysia ( fam ) , yusoff mahadi lên_tiếng , khẳng_định tổ_chức này sẽ ' chiến_đấu ' tới cùng với fifa nhằm bảo_vệ danh_tiếng của bóng_đá malaysia . nhưng ý_kiến của vị này , bị phản_đối mạnh_mẽ . 
 
 fam kêu_gọi cuộc_chiến pháp_lý với fifa , dư_luận lại can_ngăn 
 theo 
 new straits_times 
 , quyền chủ_tịch 
 fam 
 , yusoff 
 mahadi 
 vừa khẳng_định , cơ_quan này sẽ chuẩn_bị tất_cả các tài_liệu , bằng_chứng và lập_luận cần_thiết trước khi đưa vụ_việc ra tòa án trọng_tài thể_thao ( 
 cas 
 ) . 
 đội_tuyển malaysia sắp bị xử thua trận gặp đội_tuyển việt nam ngày 10.6 tại vòng_loại asian cup 2027 với tỷ_số 0 - 3 , dù đã thắng 4 - 0 do sử_dụng cầu_thủ không đủ điều_kiện thi_đấu và đã bị fifa xử_phạt 
 ảnh : ngọc_linh 
 " fam sẽ chuẩn_bị sẵn_sàng tất_cả các tài_liệu và thông_tin liên_quan để trình lên cas nhằm lật_ngược hình_phạt do fifa áp_đặt . các luật_sư được chỉ_định sẽ xem_xét mọi thứ cần_thiết , và fam sẽ không im_lặng

In [20]:
labels_list = [
    "thể_thao",
    "thế_giới",
    "giáo_dục",
    "kinh_tế",
    "chính_trị",
    "sức_khỏe",
    "thời_sự"
]

labels_mapping = {label: i for i, label in enumerate(labels_list)}

print("Label mapping:")
print(labels_mapping)

df['label_encoded'] = df['main_tag_tokenized'].map(labels_mapping)

# Ensure all tags is appeared
nan_count = df['label_encoded'].isnull().sum()

if nan_count > 0:
    print(f"NaN: {nan_count}")
    all_labels_in_data = set(df['main_tag'].unique())
    label_in_map = set(labels_mapping.keys())
    unmapped_labels = all_labels_in_data - label_in_map

print("\nAfter encoding:")
print(df[['main_tag', 'label_encoded']].head(10))

print("\nTags distribution check:")
print(df['label_encoded'].value_counts().sort_index())


Label mapping:
{'thể_thao': 0, 'thế_giới': 1, 'giáo_dục': 2, 'kinh_tế': 3, 'chính_trị': 4, 'sức_khỏe': 5, 'thời_sự': 6}

After encoding:
   main_tag  label_encoded
0  Thể thao              0
1  Thể thao              0
2  Thể thao              0
3  Thể thao              0
4  Thể thao              0
5  Thể thao              0
6  Thể thao              0
7  Thể thao              0
8  Thể thao              0
9  Thể thao              0

Tags distribution check:
label_encoded
0    3032
1    2989
2    2987
3    2974
4    2959
5    2953
6    2883
Name: count, dtype: int64


In [21]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['label_encoded', 'main_tag', 'main_tag_standardized', 'main_tag_cleaned', 'main_tag_tokenized'])
y = df['label_encoded']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print(f"Sum of starting samples: {len(df)}")
print("-" * 30)
print(f"The number of samples in train: {len(X_train)} ({len(X_train)/len(df):.0%})")
print(f"The number of samples in validation: {len(X_val)} ({len(X_val)/len(df):.0%})")
print(f"The number of samples in test: {len(X_test)} ({len(X_test)/len(df):.0%})")

print("\nCheck the tag distribution in train:")
print(y_train.value_counts(normalize=True).sort_index())

print("\nCheck the tag distribution in validation:")
print(y_val.value_counts(normalize=True).sort_index())

Sum of starting samples: 20777
------------------------------
The number of samples in train: 16621 (80%)
The number of samples in validation: 2078 (10%)
The number of samples in test: 2078 (10%)

Check the tag distribution in train:
label_encoded
0    0.145960
1    0.143854
2    0.143794
3    0.143132
4    0.142410
5    0.142109
6    0.138740
Name: proportion, dtype: float64

Check the tag distribution in validation:
label_encoded
0    0.145813
1    0.143888
2    0.143407
3    0.143407
4    0.142445
5    0.141963
6    0.139076
Name: proportion, dtype: float64


In [22]:
import os

os.makedirs('train', exist_ok=True)
os.makedirs('val', exist_ok=True)
os.makedirs('test', exist_ok=True)

X_train.to_csv('./train/X_train_basic.csv', index=False, encoding='utf-8')
y_train.to_frame().to_csv('./train/y_train_basic.csv', index=False, encoding='utf-8')

X_val.to_csv('./val/X_val_basic.csv', index=False, encoding='utf-8')
y_val.to_frame().to_csv('./val/y_val_basic.csv', index=False, encoding='utf-8')

X_test.to_csv('./test/X_test_basic.csv', index=False, encoding='utf-8')
y_test.to_frame().to_csv('./test/y_test_basic.csv', index=False, encoding='utf-8')

print("\nCreated final dataset.")


Created final dataset.


In [23]:
X_train_loaded = pd.read_csv('./train/X_train_basic.csv')
print("X_train head():")
print(X_train_loaded.head())

y_train_df = pd.read_csv('./train/y_train_basic.csv')
print("\ny_train (changed to Series) head():")
print(y_train_df.head())

X_train head():
                                                link        publication_date  \
0  https://thanhnien.vn/hang-cong-ghi-ban-dang-ca...  19/08/2025 18:38 GMT+7   
1  https://thanhnien.vn/phat-huy-dan-chu-dua-vao-...  06/09/2024 06:19 GMT+7   
2  https://thanhnien.vn/tap-doan-hanwha-tai-tro-g...  04/11/2025 08:00 GMT+7   
3  https://thanhnien.vn/lu-song-cau-rut-cham-lu-s...  08/10/2025 13:31 GMT+7   
4  https://thanhnien.vn/bao-so-9-du-bao-do-bo-qua...  24/09/2025 07:06 GMT+7   

                                               title  \
0  Hàng công ghi bàn đẳng cấp thắng Thái Lan, đội...   
1  Phát huy dân chủ, dựa vào dân xây dựng Đảng, N...   
2  Tập đoàn Hanwha tài trợ giải golf quốc tế Hanw...   
3  Lũ sông Cầu rút chậm, lũ sông Thương sắp vượt ...   
4  Bão số 9 dự báo đổ bộ Quảng Ninh - Hưng Yên, g...   

                                             content      source  \
0  Với sự tỏa sáng của Hải Yến, Huỳnh Như và Bích...  Thanh Niên   
1  Ngày 5.9, tạiHà Nội, Ủy ban